## Nicole's updates
+ cleaned the data
+ preprocessed by creating ratings matrix
+ Implemented ALS with user and item bias(fast matrix solutions)
+ Soon adding temporality to the mix
+ Had run into error that was causing singular matrix error
+ last cell has the previous implementation

In [3]:
import pandas as pd
import numpy as np
#read in the data
data = pd.read_csv("../data/ml-latest-small/ratings.csv")

#get the necessary dimensions
n_users = data['userId'].nunique()
n_movies = data['movieId'].nunique()

In [4]:
#split the data into different bins for the temporal function
def get_bin(num):
    #subtracts 25 because they started collecting the data 25 years after january 1970
    return int((num)/(60*60*24*365)) - 25
data["bin"] = data["timestamp"].apply(get_bin)
data_groups = data.groupby("bin")

In [8]:
#create R matrix
n_users = data.userId.unique().shape[0]
n_items = data.movieId.unique().shape[0]
movieIds = sorted(data.movieId.unique())

R_init = np.zeros((n_users, n_items))
bin_match = np.zeros((n_users, n_items)) 

#stored the bin number for each user movie pair
for row in data.itertuples():
    #print(row)
    R_init[row[1]-1, movieIds.index(row[2])] = row[3]
    bin_match[row[1]-1, movieIds.index(row[2])] = row[5]-1

In [6]:
#get initial B_i_bin_t dimensions: n_bins by n_movies
B_i_bint = np.zeros((data['bin'].nunique(),n_movies))
for i in range(len(B_i_bint)):
    sub_frame = data[data.bin==i+1]
    #for movie in movieId, then access 
    count = 0
    total = 0
    for j in range(len(movieIds)):
        r = sub_frame['rating'][sub_frame['movieId']==movieIds[j]].values
        B_i_bint[i,j] = (r.sum())/((r!=0).sum()).astype(float)
B_i_bint  = B_i_bint - (R_init.sum()/(R_init!=0).sum().astype(float))
B_i_bint[np.isnan(B_i_bint)] = 0   
    

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in double_scalars
  # Remove the CWD from sys.path while we load stuff.


In [14]:
def get_error(R, R_hat):
    #only for the that had ratings in the training matrix
    mask = (R>0)
    R_hat[mask] = 0 #set the values we don't need predictions for to 0 
    return np.sqrt(((R - R_hat)**2).mean()) #return the RMSE

def predict(U, M, mu,b_i, b_u, bin_match ,B_i_bint):
    R = np.zeros((n_users, n_movies))
    #do we need to iterate over time as well
    for u in range(n_users):#iterate over users
        for i in range(n_movies):#iterate over movies
            bin_num = int(bin_match[u,i]-1) #get the bin number for user u and movie i
            R[u,i] = mu+b_u[u]+b_i[i]+B_i_bint[bin_num,i]+np.dot(U[u,:],M[i,:].T)
            #R[u,i] = b_u[u]+b_i[i]+np.dot(U[u,:],M[i,:].T)
    return R

In [15]:
#basic ALS + some biases
import copy 
np.random.seed(1)
def train(Mat, bin_match, f, lambda_, n_iter, B_i_bint):
    R = Mat[:]
    # Step 1 Initialize matrix M by assigning the average rating for that movie as the first row, 
    # and small random numbers for the remaining entries.
    M = np.random.rand(n_movies,f)
    M[:,0] = R.sum(0)/(R!=0).sum(0).astype(float)
    
    #Randomly initialize U matrix
    U = np.random.rand(n_users,f)
    
    mu = R.sum()/(R!=0).sum().astype(float)#average rating
    b_i = np.random.rand(n_movies) - mu #item bias len(movies)
    b_u = np.random.rand(n_users) - mu #user bias len(users)

    #function to minimize: sum(r_ui - ^r_ui) + lambda(b_i^2 + b_u^2+ |q|^2 + |p|^2 )
    for epoch in range(n_iter):
        #Step 2 Fix M, Solve U by minimizing the objective function
        for u in range(n_users):
            E = np.identity(f+1)
            m = np.ones((n_movies,1))
            r_u = R[u,:] - b_i - mu
            M_p = np.concatenate((m,M),1)
            MTM_prime = np.dot(M_p.T, M_p)
            u_prime = np.linalg.solve((MTM_prime+(lambda_*E)), np.dot(M_p.T, r_u))
            b_u[u] = u_prime[0]
            U[u,:] = u_prime[1:]
            
        #Step 3 Fix U, solve M by minimizing the objective function similarly; 
        for m in range(n_movies):
            u = np.ones((n_users,1))
            r_m = R[:,m] - b_u - mu
            U_p = np.concatenate((u,U),1) #add 1 to each row to the data
            UTU_prime = np.dot(U_p.T, U_p)
            m_prime = np.linalg.solve((UTU_prime+(lambda_*E)), np.dot(U_p.T, r_m))
            b_i[m] = m_prime[0]
            M[m,:] = m_prime[1:]
            
        for t in range(len(B_i_bint)):
            for i in range(n_movies):
                for u in range(n_users):
                    n_it = 0
                    if bin_match[u,i] ==t:
                        B_i_bint[t,i]+= (R[u,i] - mu - b_u[u] - b_i[i] - np.dot(U[u,:], M[i,:].T))
                        if R[u,i]>0:
                            n_it+=1
                B_i_bint[t,i] = B_i_bint[t,i]/(n_it+lambda_)

        R_hat = predict(U, M, mu,b_i, b_u, bin_match ,B_i_bint)
        error = get_error(R, R_hat)
        print("iteration: ", epoch, "; error: ", error)
                   
    return U, M, mu, b_u, b_i, B_i_bint[t,i]


In [16]:
U_hat, M_hat, b_u_hat, b_i_hat, B_i_bint_hat = train(R_init, bin_match, 10, 0.4, 10, B_i_bint)

iteration:  0 ; error:  66.75464054196209
iteration:  1 ; error:  166.94063223205637
iteration:  2 ; error:  417.4087819764875
iteration:  3 ; error:  1043.582792980226
iteration:  4 ; error:  2609.0199847699664
iteration:  5 ; error:  6522.613936388982
iteration:  6 ; error:  16306.599170691352
iteration:  7 ; error:  40766.562330664055
iteration:  8 ; error:  101916.47018733063
iteration:  9 ; error:  254791.23974580562


ValueError: too many values to unpack (expected 5)

In [138]:
#basic ALS so far
np.random.seed(1)
def train(Mat, bin_match, f, lambda_, n_iter, B_i_bint):
    R = Mat[:]
    # Step 1 Initialize matrix M by assigning the average rating for that movie as the first row, and small random numbers for the remaining entries.
    M = np.random.rand(n_movies,f)
    M[:,0] = R.sum(0)/(R!=0).sum(0).astype(float)
    
    #Randomly initialize U matrix
    U = np.random.rand(n_users,f)
    
    
    M_old = M[:]
    U_old = U[:]
    #function to minimize: sum(r_ui - ^r_ui) + lambda(b_i^2 + b_u^2+ |q|^2 + |p|^2 )
    for epoch in range(n_iter):
        #Step 2 Fix M, Solve U by minimizing the objective function
        for u in range(n_users):
            E = np.identity(f)
            MTM = np.dot(M.T, M)
            U[u,:] = np.linalg.solve((MTM+(lambda_*E)), np.dot(M.T, R[u,:]))

        #Step 3 Fix U, solve M by minimizing the objective function similarly; 
        for m in range(n_movies):
            UTU = np.dot(U.T, U)
            
            M[m,:] = np.linalg.solve((UTU+(lambda_* np.identity(f))), np.dot(U.T, R[:,m]))
        
        R_hat = predict(U,M)
        error = get_error(R, R_hat)
        print("iteration: ", epoch, "; error: ", error)
    return U, M


In [139]:
U_hat, M_hat = train(R_init,bin_match, 10,2, 25, B_i_bint)

iteration:  0 ; error:  0.4798025294073136
iteration:  1 ; error:  0.5009364653350341
iteration:  2 ; error:  0.5023913869515706
iteration:  3 ; error:  0.5032194880828139
iteration:  4 ; error:  0.5037734905493885
iteration:  5 ; error:  0.5041850246011575
iteration:  6 ; error:  0.5045032663131709
iteration:  7 ; error:  0.5047542608988321
iteration:  8 ; error:  0.5049556940556682
iteration:  9 ; error:  0.5051201578127348
iteration:  10 ; error:  0.5052566436154585
iteration:  11 ; error:  0.5053716026787848
iteration:  12 ; error:  0.5054697153794211
iteration:  13 ; error:  0.5055544289540975
iteration:  14 ; error:  0.505628324985225
iteration:  15 ; error:  0.5056933691047417
iteration:  16 ; error:  0.5057510815468104
iteration:  17 ; error:  0.5058026550182755
iteration:  18 ; error:  0.5058490374194399
iteration:  19 ; error:  0.5058909908820072
iteration:  20 ; error:  0.5059291346223339
iteration:  21 ; error:  0.5059639765506403
iteration:  22 ; error:  0.5059959369330491

In [110]:
"""#alternate implementation from online resource

np.random.seed(1)
def train(Mat, bin_match, f, Lambda, n_iter, B_i_bint):
    R = Mat[:]
    # Step 1 Initialize matrix M by assigning the average rating for that movie as the first row, and small random numbers for the remaining entries.
    M = 1*np.random.rand(f, R.shape[1])
    M[0,:] = R.sum(0)/(R!=0).sum(0).astype(float)
    
    U = 1*np.random.rand(f, R.shape[0])#initialize U matrix
    
    mu = R.sum()/(R!=0).sum().astype(float)#average rating
    b_i = (R.sum(0)/(R!=0).sum(0).astype(float)) - mu #initialize b_i = len(movies)
    b_u = (R.sum(1)/(R!=0).sum(1).astype(float)) - mu #initialize b_u = len(users)
    
    #function to minimize: sum(r_ui - ^r_ui) + lambda(b_i^2 + b_u^2+ |q|^2 + |p|^2 )
    for epoch in range(n_iter):
        #Step 2 Fix M, Solve U by minimizing the objective function
        for i in range(n_users):
            I_i = np.nonzero(R[i,:])[0] #set of movies that user i rated
            n_ui = len(I_i) #number of ratings user i has given matrix 
            M_Ii = M[:,I_i] #denotes the sub-matrix of M where rows j in I_i are selected
            E = np.identity(f)
            R_i_I = R[i,I_i]
            U[:,i] = np.dot(np.linalg.inv(np.dot(M_Ii,M_Ii.T) + (Lambda*n_ui*E)),np.dot(M_Ii, R_i_I.T))

        #Step 3 Fix U, solve M by minimizing the objective function similarly; 
        for j in range(n_movies):
            I_j = np.nonzero(R[:,j])[0] #set of users that have rated movie j
            n_mj = len(I_j) #number of ratings movie j has received 
            U_Ij = U[:,I_j] #denotes the sub-matrix of U where rows i in I_j are selected
            R_j_I = R[I_j, j]
            #print(U_Ij)
            M[:,j] = np.dot(np.linalg.inv(np.dot(U_Ij,U_Ij.T) + (Lambda*n_mj*E)),np.dot(U_Ij,R_j_I.T))
        
        #get predictions
        #R_hat = predict(U, M, mu,b_i, b_u, bin_match ,B_i_bint)
    
        #get the error
        #error = get_error(R_hat, R)
        #print(error)
        
        #b_i = b_i +  
        #update all the biases
        
    #Step 4 Repeat Steps 2 and 3 until a stopping criterion is satisfied.
    return R, U, M, b_u, b_i, B_i_bint"""
#attempted implementation according to the paper. Runs into singulatity problems

'#alternate implementation from online resource\n\nnp.random.seed(1)\ndef train(Mat, bin_match, f, Lambda, n_iter, B_i_bint):\n    R = Mat[:]\n    # Step 1 Initialize matrix M by assigning the average rating for that movie as the first row, and small random numbers for the remaining entries.\n    M = 1*np.random.rand(f, R.shape[1])\n    M[0,:] = R.sum(0)/(R!=0).sum(0).astype(float)\n    \n    U = 1*np.random.rand(f, R.shape[0])#initialize U matrix\n    \n    mu = R.sum()/(R!=0).sum().astype(float)#average rating\n    b_i = (R.sum(0)/(R!=0).sum(0).astype(float)) - mu #initialize b_i = len(movies)\n    b_u = (R.sum(1)/(R!=0).sum(1).astype(float)) - mu #initialize b_u = len(users)\n    \n    #function to minimize: sum(r_ui - ^r_ui) + lambda(b_i^2 + b_u^2+ |q|^2 + |p|^2 )\n    for epoch in range(n_iter):\n        #Step 2 Fix M, Solve U by minimizing the objective function\n        for i in range(n_users):\n            I_i = np.nonzero(R[i,:])[0] #set of movies that user i rated\n          

In [112]:
np.sum(R_init>0)

100836

In [113]:
R_init.shape[0] * R_init.shape[1]

5931640